In [ ]:
!pip install pyarrow 
!pip install sqlparse
!pip uninstall --yes autoai-libs tensorflow-text numba numpy
!pip install numpy==1.22.4
!pip install --upgrade ibmcloudsql

In [ ]:
# REPLACE THIS SECTION WITH THE OUTPUT FROM:
# terraform output jupyter_notebook_configuration_python
#apikey='your apikey'
#instancecrn='your crn to the data engine instance'
#dataengineurl='cos://s3.<region>.cloud-object-storage.appdomain.cloud/logarchive-data-engine-001'
#logsurl="cos://s3.us-south.cloud-object-storage.appdomain.cloud/logarchive-001"

In [ ]:
import ibmcloudsql
import sqlparse 
from pygments import highlight
from pygments.lexers import get_lexer_by_name
from pygments.formatters import HtmlFormatter, Terminal256Formatter
lexer = get_lexer_by_name("sql", stripall=True)
formatter = Terminal256Formatter(style='vim')

In [ ]:
sqlClient = ibmcloudsql.SQLQuery(apikey, instancecrn, client_info='notebook', target_cos_url=dataengineurl, max_concurrent_jobs=4, max_tries=3 )
#sqlClient.configure()  # use this if you want to change the API key or Data Engine CRN later
    
sqlClient.logon()
print('\nYour Data Engine web console link:\n')
sqlClient.sql_ui_link()

In [ ]:
def sql_format(sql):
    formatted_sql = sqlparse.format(sql, reindent=True, indent_tabs=True, keyword_case='upper')
    return highlight(formatted_sql, lexer, formatter)

def sql_format_print(sql):
    print('\nYour SQL statement is:\n')
    print(sql_format(sql))

def sql_into(sql, intourl):
    if " INTO " not in sql:
        return sql + f' INTO {intourl} STORED AS JSON'
    return sql

def sql_r(sql):
    global dataengineurl
    sql=sql_into(sql, dataengineurl)
    sql_format_print(sql)
    result_df = sqlClient.run_sql(sql)
    print(result_df.head(10))
    return result_df

In [ ]:
# get the oldest 10 records
result_df = sql_r(f'SELECT year, month, _source__line FROM FLATTEN({logsurl} STORED AS JSON) LIMIT 10')
result_df.head(10)

In [ ]:
# find all reecords where the outcome was not a success, note you may not have any of these
result_df = sql_r(f'SELECT year, month, _source__line FROM FLATTEN({logsurl} STORED AS JSON) WHERE _source_outcome NOT REGEXP "success" LIMIT 10')
result_df.head(10)